In [18]:
import tensorflow as tf, numpy as np, shutil, random, os, csv
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import normalize as norml

In [19]:
# define variables
train_path='./data/sub_ws300/training/'
validation_path='./data/sub_ws300/validation/'
reference_train_path='./data/sub_ws300/REFERENCE.csv'
reference_validation_path='./data/sub_ws300/REFERENCE.csv'
train_dirs=os.listdir(train_path)
train_size=len(train_dirs)

batch_size= 128
display_step = 10
timesteps = 100

n_layers = 3
n_epochs = 2000
n_classes = 2
n_units = 30
n_validation_files = 300

x_placeholder = tf.placeholder('float32', [None, batch_size, timesteps])
y_placeholder = tf.placeholder('int32', [batch_size, n_classes])

validation_x_placeholder = tf.placeholder('float32', [None, n_validation_files, timesteps])
validation_y_placeholder = tf.placeholder('int32', [n_validation_files, n_classes])

In [20]:
def get_batch(path, batch_size):
    filenames = glob.glob(os.path.join(path, '*.npy'))
    return random.sample(filenames, batch_size)

def get_batch_max_len(batch_files):
    batch_max_len=0
    for f in batch_files:
        batch_max_len=max(batch_max_len, np.load(f).shape[0])
    return batch_max_len

def get_reference(reference_path, reference):
    with open(reference_path) as csvfile:
        spamreader=csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            reference[row[0]]=row[1]

def generate_batch(path, b_size):
    if b_size == None:
        b_size = batch_size

    batch_files=get_batch(path, b_size)

    batchX=np.zeros((b_size, 151, timesteps), dtype='float32')
    i=0
    for f in batch_files:
        arr=np.load(f)
        batchX[i]=arr

    i=0
    batchY=np.zeros(b_size, dtype='int32')
    for f in batch_files:
        batchY[i] = reference_train[f.split('/')[-1].split('_')[0]]
        i += 1
    
    # reshape batchX
#     batchX = np.swapaxes(batchX, 1, 2)
#     batchX = np.swapaxes(batchX, 0, 1)
    batchX = np.swapaxes(batchX, 0, 1)
    
    # convert batchY to onehot
    y_onehot = np.zeros((b_size, 2))
    y_onehot[np.where(batchY==1)] = np.array([0,1])
    y_onehot[np.where(batchY==-1)] = np.array([1,0])
    
    print('batchX, y_onehot, sizes', batchX.shape, y_onehot.shape)
    
    return batchX, y_onehot

In [23]:
def RNN(t_input_data, v_input_data, cell_name, n_layers):
    w = tf.Variable(tf.random_normal([n_units, n_classes]))
    b = tf.Variable(tf.random_normal([n_classes]))
    
    if cell_name == 'basic':
        input_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicRNNCell(n_units, activation=tf.nn.relu) for _ in range(n_layers)])
    elif cell_name == 'LSTM':
        input_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(n_units) for _ in range(n_layers)])
    else:
        input_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(n_units) for _ in range(n_layers)])

    with tf.variable_scope('try3'):
        t_outputs, t_states = tf.nn.dynamic_rnn(cell=input_cell, 
                                                inputs=t_input_data,
                                                dtype=tf.float32,
                                                time_major=True)
    t_pred = tf.matmul(t_outputs[-1], w) + b
    v_pred = None
    if v_input_data != None:
        v_outputs, v_states = tf.nn.dynamic_rnn(cell=input_cell, 
                                                inputs=v_input_data,
                                                dtype=tf.float32,
                                                time_major=True)
        v_pred = tf.matmul(v_outputs[-1], w) + b

    return t_pred, v_pred

def get_optimizer(opt_name, cost):
    if opt_name == 'adam':
        return tf.train.AdamOptimizer().minimize(cost)
    elif opt_name == 'ada':
        return tf.train.AdadeltaOptimizer().minimize(cost)
    elif opt_name == 'gradient':
        return tf.train.GradientDescentOptimizer(0.001).minimize(cost)
    else:
        return tf.train.MomentumOptimizer(0.001, 0.4).minimize(cost)

def train_neural_network(cell_name, layers, opt_name):
    print('Using Cell', cell_name, 'layers', layers, 'opt_name', opt_name)
    
    t_prediction, v_prediction = RNN(x_placeholder, validation_x_placeholder, cell_name, layers)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=t_prediction,labels=tf.reshape(y_placeholder, [batch_size, n_classes])))
    optimizer = get_optimizer(opt_name, cost)
    correct = tf.equal(tf.argmax(v_prediction, 1), tf.argmax(validation_y_placeholder, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float32'))
                                  
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        max_accuracy = 0
        for epoch in range(n_epochs):            
            batchX, batchY = generate_batch(train_path, None)
            if epoch % display_step == 0 or epoch == n_epochs - 1:
                validation_batchX, validation_batchY = generate_batch(validation_path, n_validation_files) 
                _, epoch_loss, epoch_validation_accuracy = sess.run([optimizer, cost, accuracy], 
                                                                    feed_dict={x_placeholder: batchX, 
                                                                        y_placeholder: batchY, 
                                                                        validation_x_placeholder: validation_batchX, 
                                                                        validation_y_placeholder: validation_batchY})
                max_accuracy = max(max_accuracy, epoch_validation_accuracy)
                print('Epoch', epoch+1, 'loss:', epoch_loss, 'Validation Accuracy:', epoch_validation_accuracy, 'max_accuracy', max_accuracy)
            else:
                _, epoch_loss = sess.run([optimizer, cost], feed_dict={x_placeholder: batchX, 
                                                                        y_placeholder: batchY})

In [24]:
# read in reference train/validation file
reference_train, reference_validation = {}, {}
get_reference(reference_train_path, reference_train)
get_reference(reference_validation_path, reference_validation)

train_neural_network('LSTM', n_layers, 'gradient')

Using Cell LSTM layers 3 opt_name gradient
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1 loss: 0.874286 Validation Accuracy: 0.473333 max_accuracy 0.473333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 11 loss: 0.764347 Validation Accuracy: 0.523333 max_accuracy 0.523333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, size

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 141 loss: 0.600879 Validation Accuracy: 0.463333 max_accuracy 0.53
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128,

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 271 loss: 0.617036 Validation Accuracy: 0.473333 max_accuracy 0.55
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 281 loss: 0.545583 Validation Accuracy: 0.46 max_accuracy 0.55
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 411 loss: 0.526415 Validation Accuracy: 0.503333 max_accuracy 0.55
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128,

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 541 loss: 0.624303 Validation Accuracy: 0.54 max_accuracy 0.55
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 551 loss: 0.593874 Validation Accuracy: 0.506667 max_accuracy 0.55
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (

...

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1351 loss: 0.670275 Validation Accuracy: 0.47 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1361 loss: 0.678809 Validation Accuracy: 0.5 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, si

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1491 loss: 0.648353 Validation Accuracy: 0.513333 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) 

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1621 loss: 0.598292 Validation Accuracy: 0.493333 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1631 loss: 0.638325 Validation Accuracy: 0.473333 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_o

Epoch 1751 loss: 0.51517 Validation Accuracy: 0.523333 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1761 loss: 0.587102 Validation Accuracy: 0.48 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot

batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 300, 100) (300, 2)
Epoch 1891 loss: 0.624299 Validation Accuracy: 0.536667 max_accuracy 0.563333
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) (128, 2)
batchX, y_onehot, sizes (151, 128, 100) 